# 査読を完了する
査読完了時に実行するタスクです。<br>
論文がアクセプトされた、もしくはリジェクトを受けて投稿先を変更する場合に実行してください。

## 査読者の削除
論拠データの公開のため、査読用プロジェクトを作成していた場合は、査読者のアカウントをメンバーから削除してください。<br>
査読用プロジェクトを作成していない場合は次のセルを実行せず、サブフローメニューに戻ってください。<br>
査読用PJの削除は任意で行って下さい。

In [ ]:
# 査読用PJの「メンバー」を表示する
import os
import traceback

from IPython.core.display import Javascript
from IPython.display import display, clear_output
import pandas as pd
import panel as pn
from requests.exceptions import RequestException

from library.task_director import TaskDirector
from library.utils.config import connect as con_config
from library.utils.config import message as msg_config
from library.utils.error import (UnauthorizedError, ProjectNotExist,
                                UnusableVault, RepoPermissionError)
from library.utils.html.button import create_button
from library.utils.input import get_grdm_connection_parameters
from library.utils.storage_provider import grdm
from library.utils.widgets import MessageBox


notebook_name = 'complete_review.ipynb'


class CollaboratorManager(TaskDirector):
    """共同研究者の情報を表示するクラスです。

    Attributes:
        instance:
            _form_box (pn.WidgetBox) : フォームを格納する。
            _msg_output (MessageBox) : ユーザーに提示するメッセージを格納する。
            token(str):パーソナルアクセストークン
            project_id(str):プロジェクトID
            remote_path(str):リモート先のパス
            grdm_url(str):GRDMのURL
            grdm(Grdm):grdmファイルのGrdmクラス
    """

    def __init__(self, working_path: str) -> None:
        """CollaboratorManager コンストラクタのメソッドです。

        Args:
            working_path(str):実行Notebookファイルパス
        """
        super().__init__(working_path, notebook_name)

        # フォームボックス
        self._form_box = pn.WidgetBox()
        self._form_box.width = 900
        # メッセージ用ボックス
        self._msg_output = MessageBox()
        self._msg_output.width = 900

        self.grdm_url = con_config.get('GRDM', 'BASE_URL')
        self.grdm = grdm.Grdm()

    def get_grdm_params(self) -> tuple[str, str]:
        """GRDMのトークンとプロジェクトIDを取得するメソッドです。

        Returns:
            str:GRDMのトークンの値を返す。
            str:プロジェクトIDの値を返す。

        """
        token = ""
        project_id = ""
        try:
            token, project_id = get_grdm_connection_parameters(self.grdm_url)
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except RepoPermissionError:
            message = msg_config.get('form', 'insufficient_permission')
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except ProjectNotExist as e:
            self._msg_output.update_error(str(e))
            self.log.error(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)
        return token, project_id

    @TaskDirector.task_cell("1")
    def generate_collaborator_section(self):
        """取得したデータから共同研究者を表示するメソッドです。"""

        self.doing_task()
        # フォーム定義
        try:
            self.token, self.project_id = self.get_grdm_params()
            clear_output()
            if not self._msg_output.has_message():
                self._form_box.clear()
                # コンポーネントのプロジェクトの共同研究者を表示
                self.generate_component_section()

        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self._msg_output.update_warning(message)
            self.log.warning(f'{message}\n{traceback.format_exc()}')
        except ProjectNotExist:
            message = msg_config.get('form', 'project_id_not_exist').format(self.project_id)
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)

        self.done_task()
        # フォーム表示
        pn.extension('tabulator')
        form_section = pn.WidgetBox()
        form_section.append(self._form_box)
        form_section.append(self._msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    def generate_component_section(self):
        """コンポーネントのプロジェクトの共同研究者を表示するメソッドです。"""
        components = self.grdm.get_component_list(self.grdm_url, self.token, self.project_id)

        if not components:
            self._msg_output.update_warning(msg_config.get('publish_argument_data_to_reviewers', 'no_component'))
            return

        for component_id, component_title in components.items():
            # プロジェクトのフォーム
            contents = self.grdm.get_collaborator_list(self.grdm_url, self.token, component_id)
            table = self.display_table(contents)
            button = self.generate_button(component_id)

            component_form_box = pn.WidgetBox()
            component_form_box.width = 900
            # フォーム定義
            title = msg_config.get('manage_collaborators', 'table_title')
            component_form_box.append(f'# {title}({component_title})')
            component_form_box.append(table)
            component_form_box.append(button)

            self._form_box.append(component_form_box)

    def generate_button(self, project_id: str) -> pn.pane.HTML:
        """メンバー一覧のページを表示するボタンを作成するメソッドです。

        Args:
            project_id(str):プロジェクトID

        Returns:
            pn.pane.HTML:ボタンを表示させる。
        """
        button_width = 500
        obj = create_button(
            url=self.grdm.build_collaborator_url(self.grdm_url, project_id),
            msg=msg_config.get('manage_collaborators', 'go_grdm'),
            target='_blank',
            button_width=f'{button_width}px'
        )
        return pn.pane.HTML(obj, width=button_width)

    def display_table(self, contents: dict) -> pn.widgets.Tabulator:
        """共同研究者の持つ情報を出力するメソッドです。

        共同研究者の名前や管理者、書き込み、読み取りの内、どの権限を持っているかなどを出力する。

        Args:
            contents(dict):共同管理者の情報

        Returns:
            pn.widgets.Tabulator:共同研究者の情報を表示する表
        """
        display_permission = {
                "admin": msg_config.get('manage_collaborators', 'admin'),
                "write": msg_config.get('manage_collaborators', 'write'),
                "read": msg_config.get('manage_collaborators', 'read')
            }

        name_list = []
        permission_list = []
        for full_name, permission in contents.items():
            name_list.append(full_name)
            permission_list.append(display_permission[permission])

        df = pd.DataFrame({
            msg_config.get('manage_collaborators', 'name_title'): name_list,
            msg_config.get('manage_collaborators', 'permission_title'): permission_list
        })
        return pn.widgets.Tabulator(
            df,
            show_index=False,
            header_align='center',
            text_align='center',
            configuration={'columnDefaults': {'headerSort': False}},
            editors={
                msg_config.get('manage_collaborators', 'name_title'): None,
                msg_config.get('manage_collaborators', 'permission_title'): None
            }
        )

CollaboratorManager(working_path=os.path.abspath('__file__')).generate_collaborator_section()

## サブフローメニューを表示する

In [ ]:
# サブフローメニューを表示する
import os
from library.task_director import TaskDirector

script_file_name = "complete_review"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()